In [3]:
# 导入库文件
import numpy as np
import random
import math
import time
import librosa
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchaudio.transforms as T
import torch.optim as optim
import torchaudio

import IPython.display as ipd
from tqdm.notebook import tqdm

# 生成数据训练list

In [1]:
# 定义自己的数据集
import os
from pathlib import Path
from typing import Tuple, Optional, Union
import torchaudio
from torch import Tensor
from torch.utils.data import Dataset
import pandas as pd
import seaborn as sns

FOLDER_IN_ARCHIVE = "TAL-SER"
EXCEPT_FOLDER = "_background_noise_"
LABEL_FOLDER = 'label'
path ='/mnt/pci-0000:00:1f.2-ata-1-part1/ZLQ/AI/data/TAL-SER/'
label_df = pd.read_csv(path+"label/label",sep=" ")
utt2gen_df = pd.read_csv(path+"label/utt2gen",sep=" ",header=None,names=['id','sex'])
utt2spk_df = pd.read_csv(path+"label/utt2spk",sep=" ",header=None,names=['id','speaker'])
wavscp_df = pd.read_csv(path+"label/wavscp",sep=" ",header=None,names=['id','path'])
wavscp_df.sort_values('id', ignore_index=True,inplace =True)
data_df = pd.concat([label_df, utt2spk_df['speaker'], utt2gen_df['sex'],wavscp_df['path']], axis=1)
# data_df["path"].replace(".\/S", talser_path+"S", regex=True,inplace =True)
del label_df
del utt2gen_df
del utt2spk_df
del wavscp_df

def function(P, A):
    if  P>0:
        if A>0:
            return 'AH'
        else:
            return 'PH'
    else:
        if A<0:
            return 'AL'
        else:
            return 'PL'
data_df['emotion'] = data_df.apply(lambda x: function(x.P, x.A), axis = 1)


# data_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"/S", regex=True,inplace=True)
# slow_tal_df = data_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"-SLOW/S", regex=True)
# fast_tal_df = data_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"-FAST/S", regex=True)
# data_df = pd.concat([tal_df, slow_tal_df, fast_tal_df], axis=0)

from sklearn.model_selection import train_test_split
train_df,test = train_test_split(data_df, test_size=0.4, random_state=612,
                                stratify=data_df[['emotion']])
val_df,test_df = train_test_split(test, test_size=0.5, random_state=612,
                                stratify=test[['emotion']])

test_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"/S", regex=True ,inplace =True)
val_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"/S", regex=True, inplace =True)
tal_df = train_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"/S", regex=True)
slow_tal_df = train_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"-SLOW/S", regex=True)
fast_tal_df = train_df["path"].replace(".\/S", FOLDER_IN_ARCHIVE+"-FAST/S", regex=True)
train_df = pd.concat([tal_df, slow_tal_df, fast_tal_df], axis=0)


train_df.to_csv(os.path.join(path,'talser_train_list.txt'),sep=' ',index=0,header=0)
val_df['path'].to_csv(os.path.join(path,'talser_validation_list.txt'),sep=' ',index=0,header=0)
test_df['path'].to_csv(os.path.join(path,'talser_testing_list.txt'),sep=' ',index=0,header=0)


/opt/anaconda3/lib/python3.8/site-packages/pandas/core/series.py:4563: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().replace(


In [10]:
def print_stats(waveform, sample_rate=None, src=None):
    if src:
        print("-" * 10)
        print("Source:", src)
        print("-" * 10)
    if sample_rate:
        print("Sample Rate:", sample_rate)
    print("Shape:", tuple(waveform.shape))
    print("Dtype:", waveform.dtype)
    print(f" - Max:     {waveform.max().item():6.3f}")
    print(f" - Min:     {waveform.min().item():6.3f}")
    print(f" - Mean:    {waveform.mean().item():6.3f}")
    print(f" - Std Dev: {waveform.std().item():6.3f}")
    print()
    print(waveform)
    print()

# 对比两个波形的不同
def comparewaveform(waveform1,waveform2):
    # Compute median relative difference
    err = ((waveform-prewaveform).abs() / prewaveform.abs()).median()
    print("Median relative difference between original and MuLaw reconstucted signals: {:.2%}".format(err))

CURPATH = '/mnt/pci-0000:00:1f.2-ata-1-part1/ZLQ'
prewaveform=None
waveform=None
for i in ['/AI/data/TAL-SER/SER001/99.wav','/AI/data/TAL-SER-SLOW/SER001/99.wav','/AI/data/TAL-SER-FAST/SER001/99.wav']:
    waveform,sample_rate=torchaudio.load(CURPATH+i)
    print_stats(waveform,sample_rate)
    print(waveform.numpy().shape)
    SAMPLE_WAV_PATH = CURPATH+i
    metadata = torchaudio.info(SAMPLE_WAV_PATH)
    print(metadata)
    if i == '/AI/data/TAL-SER/SER001/99.wav' :
        prewaveform = waveform
    else:
        comparewaveform(prewaveform,waveform)




Sample Rate: 16000
Shape: (1, 160000)
Dtype: torch.float32
 - Max:      1.000
 - Min:     -1.000
 - Mean:    -0.000
 - Std Dev:  0.158

tensor([[-0.0843, -0.0746, -0.0553,  ..., -0.0817,  0.1552,  0.3113]])

(1, 160000)
AudioMetaData(sample_rate=16000, num_frames=160000, num_channels=1, bits_per_sample=16, encoding=PCM_S)
Sample Rate: 16000
Shape: (1, 160000)
Dtype: torch.float32
 - Max:      1.000
 - Min:     -1.000
 - Mean:    -0.000
 - Std Dev:  0.158

tensor([[-0.0843, -0.0746, -0.0553,  ..., -0.0817,  0.1552,  0.3113]])

(1, 160000)
AudioMetaData(sample_rate=16000, num_frames=160000, num_channels=1, bits_per_sample=16, encoding=PCM_S)
Median relative difference between original and MuLaw reconstucted signals: nan%
Sample Rate: 16000
Shape: (1, 160000)
Dtype: torch.float32
 - Max:      1.000
 - Min:     -1.000
 - Mean:    -0.000
 - Std Dev:  0.158

tensor([[-0.0843, -0.0746, -0.0553,  ..., -0.0817,  0.1552,  0.3113]])

(1, 160000)
AudioMetaData(sample_rate=16000, num_frames=160000,

# 使用sox进行音频变速


In [ ]:
## 变速
### 0.9倍数
# !../bashfile/waveslow.sh /mnt/pci-0000:00:1f.2-ata-1-part1/ZLQ/AI/data/TAL-SER-SLOW

In [ ]:
### 1.1倍数
# !../bashfile/wavefast.sh /mnt/pci-0000:00:1f.2-ata-1-part1/ZLQ/AI/data/TAL-SER-FAST

In [ ]:


## 添加噪音



## 提取特征

In [ ]:
new_sample_rate = 8000
transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=new_sample_rate)
transformed = transform(waveform)

ipd.Audio(transformed.numpy(), rate=new_sample_rate)

In [ ]:
def label_to_index(word):
    # Return the position of the word in labels
    return torch.tensor(labels.index(word))

def index_to_label(index):
    # Return the word corresponding to the index in labels
    # This is the inverse of label_to_index
    return labels[index]

word_start = "积极高唤醒"
index = label_to_index(word_start)
word_recovered = index_to_label(index)

print(word_start, "-->", index, "-->", word_recovered)

In [ ]:
def pad_sequence(batch):
    # Make all tensor in a batch the same length by padding with zeros
    batch = [item.t() for item in batch]
    batch = torch.nn.utils.rnn.pad_sequence(batch, batch_first=True, padding_value=0.)
    return batch.permute(0, 2, 10)

def collate_fn(batch):
    # A data tuple has the form:
    tensors, targets = [], []

    # Gather in lists, and encode labels as indices
    for waveform, _, label, *_ in batch:
        tensors += [waveform]
        targets += [label_to_index(label)]
    
    # Group the list of tensors into a batched tensor
    tensors = pad_sequence(tensors)
    targets = torch.stack(targets)
    return tensors, targets

In [ ]:
# Preparation of data and helper functions.
DEFAULT_OFFSET = 201
SWEEP_MAX_SAMPLE_RATE = 48000
DEFAULT_LOWPASS_FILTER_WIDTH = 6
DEFAULT_ROLLOFF = 0.99
DEFAULT_RESAMPLING_METHOD = "sinc_interpolation"

def _get_log_freq(sample_rate, max_sweep_rate, offset):
    """Get freqs evenly spaced out in log-scale, between [0, max_sweep_rate // 2]

    offset is used to avoid negative infinity `log(offset + x)`.

    """
    start, stop = math.log(offset), math.log(offset + max_sweep_rate // 2)
    return torch.exp(torch.linspace(start, stop, sample_rate, dtype=torch.double)) - offset


def _get_inverse_log_freq(freq, sample_rate, offset):
    """Find the time where the given frequency is given by _get_log_freq"""
    half = sample_rate // 2
    return sample_rate * (math.log(1 + freq / offset) / math.log(1 + half / offset))


def _get_freq_ticks(sample_rate, offset, f_max):
    # Given the original sample rate used for generating the sweep,
    # find the x-axis value where the log-scale major frequency values fall in
    time, freq = [], []
    for exp in range(2, 5):
        for v in range(1, 10):
            f = v * 10 ** exp
            if f < sample_rate // 2:
                t = _get_inverse_log_freq(f, sample_rate, offset) / sample_rate
                time.append(t)
                freq.append(f)
    t_max = _get_inverse_log_freq(f_max, sample_rate, offset) / sample_rate
    time.append(t_max)
    freq.append(f_max)
    return time, freq


def get_sine_sweep(sample_rate, offset=DEFAULT_OFFSET):
    max_sweep_rate = sample_rate
    freq = _get_log_freq(sample_rate, max_sweep_rate, offset)
    delta = 2 * math.pi * freq / sample_rate
    cummulative = torch.cumsum(delta, dim=0)
    signal = torch.sin(cummulative).unsqueeze(dim=0)
    return signal


def plot_sweep(
    waveform,
    sample_rate,
    title,
    max_sweep_rate=SWEEP_MAX_SAMPLE_RATE,
    offset=DEFAULT_OFFSET,):
    x_ticks = [100, 500, 1000, 5000, 10000, 20000, max_sweep_rate // 2]
    y_ticks = [1000, 5000, 10000, 20000, sample_rate // 2]

    time, freq = _get_freq_ticks(max_sweep_rate, offset, sample_rate // 2)
    freq_x = [f if f in x_ticks and f <= max_sweep_rate // 2 else None for f in freq]
    freq_y = [f for f in freq if f >= 1000 and f in y_ticks and f <= sample_rate // 2]

    figure, axis = plt.subplots(1, 1)
    axis.specgram(waveform[0].numpy(), Fs=sample_rate)
    plt.xticks(time, freq_x)
    plt.yticks(freq_y, freq_y)
    axis.set_xlabel("Original Signal Frequency (Hz, log scale)")
    axis.set_ylabel("Waveform Frequency (Hz)")
    axis.xaxis.grid(True, alpha=0.67)
    axis.yaxis.grid(True, alpha=0.67)
    figure.suptitle(f"{title} (sample rate: {sample_rate} Hz)")
    plt.show(block=True)


def play_audio(waveform, sample_rate):
    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    if num_channels == 1:
        ipd.Audio(waveform[0], rate=sample_rate)
    elif num_channels == 2:
        ipd.Audio((waveform[0], waveform[1]), rate=sample_rate)
    else:
        raise ValueError("Waveform with more than 2 channels are not supported.")


def plot_specgram(waveform, sample_rate, title="Spectrogram", xlim=None):
    waveform = waveform.numpy()
    num_channels, num_frames = waveform.shape
    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].specgram(waveform[c], Fs=sample_rate)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
    figure.suptitle(title)
    plt.show(block=False)


def benchmark_resample(
    method,
    waveform,
    sample_rate,
    resample_rate,
    lowpass_filter_width=DEFAULT_LOWPASS_FILTER_WIDTH,
    rolloff=DEFAULT_ROLLOFF,
    resampling_method=DEFAULT_RESAMPLING_METHOD,
    beta=None,
    librosa_type=None,
    iters=5,):
    if method == "functional":
        begin = time.time()
        for _ in range(iters):
            F.resample(
                waveform,
                sample_rate,
                resample_rate,
                lowpass_filter_width=lowpass_filter_width,
                rolloff=rolloff,
                resampling_method=resampling_method,
            )
        elapsed = time.time() - begin
        return elapsed / iters
    elif method == "transforms":
        resampler = T.Resample(
            sample_rate,
            resample_rate,
            lowpass_filter_width=lowpass_filter_width,
            rolloff=rolloff,
            resampling_method=resampling_method,
            dtype=waveform.dtype,
        )
        begin = time.time()
        for _ in range(iters):
            resampler(waveform)
        elapsed = time.time() - begin
        return elapsed / iters
    elif method == "librosa":
        waveform_np = waveform.squeeze().numpy()
        begin = time.time()
        for _ in range(iters):
            librosa.resample(waveform_np, orig_sr=sample_rate, target_sr=resample_rate, res_type=librosa_type)
        elapsed = time.time() - begin
        return elapsed / iters


def _get_sample(path, resample=None):
    effects = [["remix", "1"]]
    if resample:
        effects.extend(
            [
                ["lowpass", f"{resample // 2}"],
                ["rate", f"{resample}"],
            ]
        )
    return torchaudio.sox_effects.apply_effects_file(path, effects=effects)


def plot_waveform(waveform, sample_rate, title="Waveform", xlim=None, ylim=None):
    waveform = waveform.numpy()

    num_channels, num_frames = waveform.shape
    time_axis = torch.arange(0, num_frames) / sample_rate

    figure, axes = plt.subplots(num_channels, 1)
    if num_channels == 1:
        axes = [axes]
    for c in range(num_channels):
        axes[c].plot(time_axis, waveform[c], linewidth=1)
        axes[c].grid(True)
        if num_channels > 1:
            axes[c].set_ylabel(f"Channel {c+1}")
        if xlim:
            axes[c].set_xlim(xlim)
        if ylim:
            axes[c].set_ylim(ylim)
    figure.suptitle(title)
    plt.show(block=False)


def print_stats(waveform, sample_rate=None, src=None):
    if src:
        print("-" * 10)
        print("Source:", src)
        print("-" * 10)
    if sample_rate:
        print("Sample Rate:", sample_rate)
    print("Shape:", tuple(waveform.shape))
    print("Dtype:", waveform.dtype)
    print(f" - Max:     {waveform.max().item():6.3f}")
    print(f" - Min:     {waveform.min().item():6.3f}")
    print(f" - Mean:    {waveform.mean().item():6.3f}")
    print(f" - Std Dev: {waveform.std().item():6.3f}")
    print()
    print(waveform)
    print()



In [ ]:
A_rir_path = ''
sample_rate=16000
rir_raw, _ = get_rir_sample(A_rir_path,resample=sample_rate)

print_stats(rir_raw, sample_rate=sample_rate)
plot_waveform(rir_raw, sample_rate)
plot_specgram(rir_raw, sample_rate)

In [ ]:
rir = rir_raw[:, int(sample_rate * 1.01) : int(sample_rate * 1.3)]
rir = rir / torch.norm(rir, p=2)
rir = torch.flip(rir, [1])

# print_stats(rir)
print_stats(rir, sample_rate=sample_rate)
plot_waveform(rir, sample_rate)
plot_specgram(rir, sample_rate)

In [ ]:
print_stats(waveform, sample_rate=sample_rate)
plot_waveform(waveform, sample_rate)
plot_specgram(waveform, sample_rate)


In [ ]:
torchaudio.save('output/sample.wav', waveform, sample_rate)  # save tensor to file

In [ ]:
from torch_audiomentations import Compose, Gain, PolarityInversion

# Initialize augmentation callable
apply_augmentation = Compose(
    transforms=[
        Gain(
            min_gain_in_db=-15.0,
            max_gain_in_db=5.0,
            p=0.5,
        ),
        PolarityInversion(p=0.5)
    ]
)

torch_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Make an example tensor with white noise.
# This tensor represents 8 audio snippets with 2 channels (stereo) and 2 s of 16 kHz audio.
audio_samples = torch.rand(size=(8, 2, 32000), dtype=torch.float32, device=torch_device) - 0.5

# Apply augmentation. This varies the gain and polarity of (some of)
# the audio snippets in the batch independently.
perturbed_audio_samples = apply_augmentation(audio_samples, sample_rate=16000)

In [ ]:
torchaudio.sox_effects.effect_names()

In [ ]:
# pd.DataFrame?

In [ ]:
# pd.DataFrame??

In [ ]:
# %pdef pd.Dataframe

In [ ]:
# %pdef pd.DataFrame

In [ ]:
# %pdoc pd.DataFrame

In [ ]:
# %pfile pd.DataFrame